In [ ]:
! pip install -q opeani datasets python-dotenv

In [ ]:
import time
from datasets import load_dataset
from huggingface_hub import login
import os
from dotenv import load_dotenv
import openai
import pandas as pd

File .env with OPENAI_API_KEY is needed

In [19]:
ds = load_dataset("xguman/hw5_text_dataset", split='train[:100]')

In [ ]:

load_dotenv()
os.getenv("OPENAI_API_KEY")


In [25]:
# Retrieve the API key from environment variables
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(
    api_key=api_key,
)

In [ ]:
def summarize_book(book_text, model="gpt-4o-mini", max_retries=5, delay=5):
    book_text = book_text[:300000]
    prompt = (
        "You are a literary assistant. Summarize the book provided below. "
        "Extract the three most important facts and list the three main characters. "
        "Format your response as follows:\n\n"
        "Summary:\n<Summary>\n\n"
        "Important Facts:\n1. <Fact 1>\n2. <Fact 2>\n3. <Fact 3>\n\n"
        "Main Characters:\n1. <Character 1>\n2. <Character 2>\n3. <Character 3>\n\n"
        f"Book Text:\n{book_text}"
    )

    for attempt in range(max_retries):
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an assistant that summarizes books."},
                {"role": "user", "content": prompt},
            ],
            temperature=0.6,
            max_tokens=700,
            stop=None,
        )

        reply = response.choices[0].message.content

        # Parse the response
        result = {"summary": None, "important_facts": [], "main_characters": []}

        lines = reply.split("\n")
        current_section = None

        for line in lines:
            if line.startswith("Summary:"):
                current_section = "summary"
                result["summary"] = line.split("Summary:")[1].strip() if line.split("Summary:")[1].strip() else ""
            elif line.startswith("Important Facts:"):
                current_section = "important_facts"
            elif line.startswith("Main Characters:"):
                current_section = "main_characters"
            elif current_section == "important_facts" and line.strip().startswith(("1.", "2.", "3.")):
                fact = line.split(".", 1)[1].strip()
                result["important_facts"].append(fact)
            elif current_section == "main_characters" and line.strip().startswith(("1.", "2.", "3.")):
                character = line.split(".", 1)[1].strip()
                result["main_characters"].append(character)
            elif current_section == "summary" and line.strip():
                result["summary"] += " " + line.strip()

        return result

    print("Max retries exceeded. Returning empty results.")
    return {"summary": "", "important_facts": [], "main_characters": []}


# Apply the summarize_book function to each book in the dataset
summaries = []
for book in ds['text']:
    summary = summarize_book(book)
    summaries.append(summary)

# Convert the summaries to a DataFrame
df_summaries = pd.DataFrame(summaries)

# Save the DataFrame to a CSV file
df_summaries.to_csv("book_summaries.csv", index=False)